In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [ ]:
from visualize_pyphi import *
import pickle as pkl
from tqdm.auto import tqdm


In [ ]:
with open(r"example_data/ch3_subsystem.pkl", "rb") as f:
    subsystem = pkl.load(f)

In [ ]:
# get a compositional state from a system_state
compositional_state = compute.compositional_state_from_system_state(subsystem.state)
compositional_state

In [ ]:
# we can now filter a ces based on the compositional state
ces = compute.unfold_separated_ces(subsystem)
filtered_ces = compute.filter_ces_by_compositional_state(ces, compositional_state)
print('original CES had {} distinctions'.format(len(ces)))
print('filtered CES has {} distinctions'.format(len(filtered_ces)))

In [ ]:
# we can also find the compositional state based on the original CES
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

In [ ]:
# and we can filter the CES based on one of these compositional states
filtered_ces = compute.filter_ces_by_compositional_state(ces, all_compositional_states[0])
print('original CES had {} distinctions'.format(len(ces)))
print('filtered CES has {} distinctions'.format(len(filtered_ces)))

In [ ]:
# we can also get the relations and filtered CES using a specific compositional states
(
    filtered_ces,
    filtered_relations,
    compositional_state,
) = compute.compute_rels_and_ces_for_compositional_state(
    subsystem, all_compositional_states[-1], ces
)
print("original CES had {} distinctions".format(len(ces)))
print("filtered CES has {} distinctions".format(len(filtered_ces)))

In [ ]:
# if we want to be fancy, we can also pick the compositional state that maximizies (a proxy for) Big Phi
max_ces = compute.get_maximal_ces(subsystem, ces=None, max_k=3)
max_ces['compositional_state']

In [ ]:
# to avoid explosions we can also approximate the ces by picking the compositional state that maximizes small phi 
(
    filtered_ces,
    filtered_relations,
    compositional_state,
) = compute.compute_rels_and_ces_for_compositional_state(
    subsystem, all_compositional_states[-1], ces
)

In [ ]:
# and now we can plot the CES
fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/max_ces',surface_opacity=0.7,)

In [ ]:
units = [2,3]
fig = plots.compound_distinction(
    subsystem,
    max_ces['ces'],
    units,
    f'figures/compound_{units}',
    relations=max_ces['relations'],
)

In [ ]:
untouched_c, untouched_r = compute.get_untouced_ces_and_rels(max_ces['ces'],max_ces['relations'],max_ces['MIP'],)


In [ ]:
killed_d = pyphi.models.subsystem.CauseEffectStructure([mice for mice in max_ces['ces'] if mice not in untouched_c])
killed_r = [r for r in max_ces['relations'] if r not in untouched_r]

In [ ]:

cess = [max_ces['ces'], untouched_c]
relations = [max_ces['relations'], untouched_r]
nonstandard_kwargs = [
    dict(
        network_name='figures/partitioned',
        surface_colorscale='Greys',
        surface_opacity=0.1,
        show_labels=False,
        show_links=False,
        show_edges=False
    ),
    dict(
        network_name='figures/partitioned',
        surface_colorscale='plasma',
        surface_opacity=1.0,
        show_chains=False,
        show_mechanism_base=False,
    ),
]

fig = plots.overlaid_ces_plot(subsystem, cess, relations, nonstandard_kwargs)

In [ ]:
fig = plots.plot_effect_of_MIP(
    subsystem,
    max_ces['ces'],
    max_ces['relations'],
    'figures/MIP',
    partitions=None,
    common_kwargs=dict(),
    uncommon_kwargs=[dict(), dict()],
)

In [ ]:
system = subsystem
ces = max_ces["ces"]
relations = max_ces["relations"]
figure_name = "figures/component"

# Make a list of True and False showing which mices are included in your component 
# here we just make a list which is true for any mice that has some units in their mechanism or purview
units = (0,1)
component_distinctions = [
    True
    if (
        all([unit in mice.mechanism for unit in units])
        or all([unit in mice.purview for unit in units])
    )
    else False
    for mice in ces
]

fig = plots.plot_component(
    system,
    ces,
    component_distinctions,
    figure_name,
    relations=relations,
    uncommon_kwargs=[dict(), dict(), dict(save_plot_to_html=True)],
)

In [ ]:
# compute all components
components = {
    component_distinction: compute.get_component_phi(ces, relations, component_distinction)
    for component_distinction in tqdm(list(pyphi.utils.all_states(len(ces))))
}
component_phi = {key: value[0] for key, value in components.items()}
sorted_components = dict(
    sorted(component_phi.items(), key=lambda item: item[1], reverse=True)
)

for i, component_distinctions in enumerate(sorted_components.keys()):
    if i < 5:
        plots.plot_component(
            system,
            ces,
            component_distinctions,
            f'figures/component_{i}',
            relations=relations,
            uncommon_kwargs=[dict(), dict(), dict()],
        )

In [ ]:
ces = max_ces['ces']
triggered_distinctions = [True, False, False, True, False, True, False, True, 
                          True, False, False, True, True, False, True, False]

fig = plots.plot_perception(subsystem,ces,triggered_distinctions,'figures/perception')

In [ ]:
import numpy as np

net_name = "ch3"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 15
x0 = 0.7

q = 0.6
s = 0.8
l = 0.25
j = 1 - q
i = j - 0.05
b = 0.01

weights = np.array(
    [
        [s, l, 0, i, 0, 0],  # A
        [l, s, l, j, b, 0],  # B
        [0, l, s, i, 0, 0],  # C
        [b, b, b, q, 0, i],  # D
        [0, i, 0, 0, 0, 0],  # I
        [0, 0, 0, 0, 0, 0],  # O
        # A, B, C, D, I, O
    ]
)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = [f for f in "sssgll"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 1, 0, 1, 1, 1)
subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = list(compute.unfold_separated_ces(subsystem))

# repairing the broken CES
ces[-14] = subsystem.find_mice(
    pyphi.direction.Direction.CAUSE, (1, 3), purviews=tuple([(1, 2, 3)])
)
direction = pyphi.direction.Direction.CAUSE
mechanism = (0, 1, 2)
purview = (0, 1, 2)
partition = list(pyphi.partition.all_partitions((0, 1, 2), (0, 1, 2)))[32]
repertoire = subsystem.find_mice(direction, mechanism, tuple([purview])).repertoire
phi, partitioned_repertoire = subsystem.evaluate_partition(
    direction, mechanism, purview, partition
)
ria = pyphi.models.mechanism.RepertoireIrreducibilityAnalysis(
    phi,
    direction,
    mechanism,
    purview,
    partition,
    repertoire,
    partitioned_repertoire,
    node_labels=subsystem.node_labels,
)
ces[20] = pyphi.models.mechanism.MaximallyIrreducibleCauseOrEffect(ria)

ces = pyphi.models.CauseEffectStructure(ces)
max_ces = compute.get_maximal_ces(subsystem, ces=ces, max_k=3)